### Написать свой пользовательский dataset используя наследование от torch.utils.data.Dataset

Постановка.  
В общем случае, в задачах ML, мы будем иметь базу данных и файл разметки базы данных (если это обучение с учителем). На пример, если мы решаем задачу детектирования объектов, то в файле разметки для каждого изображения должны содержаться:
1. Название изображения
2. коодринаты рамок (bounding boxes) для каждого объекта на изображении
3. классы каждого изображения

Для задачи MNIST мы имеем 10 папок классов (номера цифр). Так же, имеем файл разметки digitTrain.csv (digitTest.csv). Файлы разметки содержат названия файлов и классы (есть еще колонка - angle наклона цифры, она нам сейчас не нужна). Мы хотим решить задачу классификации. Необходимо написать класс, который наследует от Dataset, на вход принимает файл разметки, на выходе для заданного индекса выдает соответствующую картинку (в формате torch.tensor) и класс изображения (onehot вектор)

In [16]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [4]:
df = pd.read_csv('DigitDataset/digitTrain.csv')
df_test = pd.read_csv('DigitDataset/digitTest.csv')
df

,image,digit,angle
0,image2119.png,3,-42
1,image2593.png,3,-15
2,image6703.png,7,-24
3,image6950.png,7,-44
4,image6462.png,7,13
...,...,...,...
4995,image7095.png,8,8
4996,image8253.png,9,39
4997,image7114.png,8,25
4998,image8841.png,9,33


Сделаем то что нам необходимо без использования классов для произвольного idx, например = 2

In [5]:
idx = 2
image = plt.imread('DigitDataset/'+str(df.digit[idx])+'/'+df.image[idx])
image.shape


Необходимо преобразовать данные в тип torch.tensor

In [6]:
image = torch.tensor(image)
print(image.shape)

torch.Size([28, 28])


Мы получили картинку размера 28х28. Torch требует, чтобы формат изображения был CxHxW (канал, высота, ширина)

In [7]:
image = image.unsqueeze(0)
image.shape

torch.Size([1, 28, 28])

Метку класса необходимо преобразовать к onehot вектору

In [19]:
labels = torch.zeros(10)
labels[int(df.digit[idx])] = 1
print(labels,int(df.digit[idx]))

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]) 7


Теперь организуем через класс imgDataset

In [8]:

class imgDataset(Dataset):
    def __init__(self,df):
        self.df = df
        #self.label = torch.zeros(10)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        image = plt.imread('DigitDataset/'+str(self.df.digit[idx])+'/'+self.df.image[idx])
        image = torch.tensor(image)
        image = image.unsqueeze(0) # первая размерность - это канал (в нашем случае один канал)
        label = int(self.df.digit[idx])  
        labels = torch.zeros(10)
        labels[label] = 1 #  Создаем onehot вектор, но можно и так labels = int(self.df.digit[idx])
        
            
        return {"image": image, "label": labels} 
    

dataset = imgDataset(df)
test_dataset = imgDataset(df_test)

In [9]:
t = next(iter(dataset))
(t['label'])

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

Dataloader

In [10]:
data_load = DataLoader(dataset,batch_size=32, shuffle=True, pin_memory=True) 
data_load_test = DataLoader(test_dataset,batch_size=len(test_dataset))

In [11]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
import torch.nn as nn
import torch.nn.functional as F

cuda


In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(1568, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)   
        output = self.out(x)
        return output, x    

In [13]:
cnn = CNN().to(device)
loss_func = nn.CrossEntropyLoss() 

In [15]:
import torch.optim as optim
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)
num_epochs = 10
for epoch in range(num_epochs):
        k = 0
        z = iter(data_load)
        for data in z:
            optimizer.zero_grad()
            X = data['image'].to(device)
            label = data['label'].to(device)
            output = cnn(X)
            loss = loss_func(output[0], label)
              
            optimizer.zero_grad()           
            
            loss.backward()    
            
            optimizer.step()                
            
            if k==0 :
                z_test = next(iter(data_load_test))
                X = data['image'].to(device)
                label = data['label'].to(device)
                with torch.no_grad():
                    output = cnn(X)
                    loss = loss_func(output[0], label)
                    print ( loss.item())
            k = 1

2.3040804862976074
2.2765660285949707
2.046933650970459
1.2539881467819214
1.1189889907836914
0.9935417175292969
1.2099921703338623
0.6574617624282837
0.761924147605896
0.29973816871643066
